## Reading and creating new table in Cassandra

In [66]:
import json
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, BatchStatement
from cassandra.query import SimpleStatement
from cassandra.auth import PlainTextAuthProvider
from cassandra import InvalidRequest as CE
import pandas as pd

## Cassandra credentials
cassandrauser = 'cassandra'
casspassword = 'cassandra'
host_list = 'localhost'
cassandra_port =9042

### Create connection to cassandra database.

def connect_cassandra():
    auth_provider = PlainTextAuthProvider(username=cassandrauser,password=casspassword)
    cluster = Cluster([host_list],protocol_version=4,
                      auth_provider = auth_provider,port=cassandra_port)
    session = cluster.connect()
    session.set_keyspace('twitter')
    return  session

cursor_twitter = connect_cassandra()

### Reading data from tweets_coronavirus to a dataframe.

In [67]:
query = "select * from tweets_coronavirus"
tweets_coronavirus = pd.DataFrame(list(cursor_twitter.execute(query)))
tweets_coronavirus.head(10)

,id,created_at,fullmessage,location,place,retweetcount,tweet,user_id
0,23440890-75d0-11ea-933c-d984957af244,Fri Apr 03 17:25:34 +0000 2020,"{""created_at"":""Fri Apr 03 17:25:34 +0000 2020""...",None,None,0,"RT @o_rips: US population 327,200,000\n\n7,452...",783040259437101057
1,233c23e0-75d1-11ea-933c-d984957af244,Fri Apr 03 17:32:43 +0000 2020,"{""created_at"":""Fri Apr 03 17:32:43 +0000 2020""...",🇳🇬,None,0,RT @todayng: Coronavirus: Didier Drogba slams ...,237738614
2,75ef0ef0-75d0-11ea-933c-d984957af244,Fri Apr 03 17:27:52 +0000 2020,"{""created_at"":""Fri Apr 03 17:27:52 +0000 2020""...","Connecticut, USA",None,0,A national outbreak calls for a national respo...,278124059
3,e4a60620-75ce-11ea-933c-d984957af244,Fri Apr 03 17:16:39 +0000 2020,"{""created_at"":""Fri Apr 03 17:16:39 +0000 2020""...","Port Mouton, NS Canada",None,0,RT @MaximeBernier: The problem with the Trudea...,951666961
4,4edf3d60-75cd-11ea-933c-d984957af244,Fri Apr 03 17:05:18 +0000 2020,"{""created_at"":""Fri Apr 03 17:05:18 +0000 2020""...","Albuquerque, New Mexico, USA",None,0,RT @RepBarbaraLee: Even Trump’s own experts ag...,24886197
5,de4c0c20-75ce-11ea-933c-d984957af244,Fri Apr 03 17:16:29 +0000 2020,"{""created_at"":""Fri Apr 03 17:16:29 +0000 2020""...","Falmouth, England",United Kingdom,0,A tide is turning. Making Health and Nutrition...,24881428
6,42e34ec0-75cd-11ea-933c-d984957af244,Fri Apr 03 17:04:58 +0000 2020,"{""created_at"":""Fri Apr 03 17:04:58 +0000 2020""...",D.M.V. (Eastern shore Md),None,0,RT @bsonenstein: here's a better idea than giv...,165452390
7,660d6200-75cd-11ea-933c-d984957af244,Fri Apr 03 17:05:57 +0000 2020,"{""created_at"":""Fri Apr 03 17:05:57 +0000 2020""...",Gród Lecha,None,0,RT @SpeedBird_NCL: The big picture: KLM fleet ...,369581565
8,942e2180-75d0-11ea-933c-d984957af244,Fri Apr 03 17:28:43 +0000 2020,"{""created_at"":""Fri Apr 03 17:28:43 +0000 2020""...",None,None,0,RT @PENamerica: Professors send open letter to...,837698190
9,e2337820-75d1-11ea-933c-d984957af244,Fri Apr 03 17:38:04 +0000 2020,"{""created_at"":""Fri Apr 03 17:38:04 +0000 2020""...","Brooklyn, NY",None,0,Damn 😔🥺,62115298


### Filter the users that have posted more than 20 tweets

In [68]:
tweets_count = pd.DataFrame(tweets_coronavirus.groupby('user_id').filter(lambda x: (x['tweet']).count() > 20 ))
tweets_count.head(5)

,id,created_at,fullmessage,location,place,retweetcount,tweet,user_id
185,280ff310-75d1-11ea-933c-d984957af244,Fri Apr 03 17:32:51 +0000 2020,"{""created_at"":""Fri Apr 03 17:32:51 +0000 2020""...","New Haven, Connecticut, USA",None,0,RT @peterbakernyt: The CIA has been warning th...,72677520
231,6c449c50-75ce-11ea-933c-d984957af244,Fri Apr 03 17:13:17 +0000 2020,"{""created_at"":""Fri Apr 03 17:13:17 +0000 2020""...",None,None,0,RT @NehaBindal10: #SelflessServiceByDSSVolunte...,1236616242544295936
390,b8454950-75cf-11ea-933c-d984957af244,Fri Apr 03 17:22:34 +0000 2020,"{""created_at"":""Fri Apr 03 17:22:34 +0000 2020""...",None,None,0,"RT @RepStefanik: Thanks to @RepGallagher, @Rep...",1175794398476427264
967,516db3a0-75d1-11ea-933c-d984957af244,Fri Apr 03 17:34:01 +0000 2020,"{""created_at"":""Fri Apr 03 17:34:01 +0000 2020""...",None,None,0,RT @BuzzFeedNews: The White House announced a ...,1238467680773894146
1136,7369a720-75d1-11ea-933c-d984957af244,Fri Apr 03 17:34:58 +0000 2020,"{""created_at"":""Fri Apr 03 17:34:58 +0000 2020""...",LA,None,0,RT @PressSec: Many American companies have ans...,49380102


In [69]:
query = "create table if not exists tweets_count(user_id text, location text, tweet text, retweet_count int,created_at text,PRIMARY KEY(user_id, created_at))"
prepared = cursor_twitter.execute(query)

In [70]:
tweets_count.columns

Index(['id', 'created_at', 'fullmessage', 'location', 'place', 'retweetcount',
       'tweet', 'user_id'],
      dtype='object')

In [71]:
query = "INSERT INTO tweets_count(user_id, location, tweet, retweet_count,created_at) VALUES (?,?,?,?,?)"
prepared = cursor_twitter.prepare(query)

In [72]:
for index, data in tweets_count.iterrows():
    cursor_twitter.execute(prepared, (data.user_id,data.location,data.tweet,data.retweetcount,data.created_at))

### Reading data for all users where they have more than 20 tweets to a dataframe.
### Creating a new table with this dataframe and write this data into a table.

In [73]:
query = "select * from tweets_count"
users_with_tweets_more_than_20 = pd.DataFrame(list(cursor_twitter.execute(query)))

users_with_tweets_more_than_20.head(10)

,user_id,created_at,location,retweet_count,tweet
0,778395329599111169,Fri Apr 03 17:16:36 +0000 2020,"Oregon, USA",0,RT @DerekCressman: People who’d dealt with Kus...
1,778395329599111169,Fri Apr 03 17:17:59 +0000 2020,"Oregon, USA",0,RT @girlsreallyrule: Here is Captain Crozier w...
2,778395329599111169,Fri Apr 03 17:22:40 +0000 2020,"Oregon, USA",0,RT @KamalaHarris: Fifty days ago I attended a ...
3,778395329599111169,Fri Apr 03 17:23:41 +0000 2020,"Oregon, USA",0,RT @glennkirschner2: Trump’s responsibility fo...
4,778395329599111169,Fri Apr 03 17:25:45 +0000 2020,"Oregon, USA",0,RT @jaketapper: Hey does anyone know anyone wh...
5,778395329599111169,Fri Apr 03 17:26:44 +0000 2020,"Oregon, USA",0,RT @SethAbramson: The only people not profound...
6,778395329599111169,Fri Apr 03 17:28:08 +0000 2020,"Oregon, USA",0,"RT @brianstelter: This is a stunning lie, and ..."
7,778395329599111169,Fri Apr 03 17:29:36 +0000 2020,"Oregon, USA",0,"RT @neeratanden: Over 6,000 Americans have die..."
8,778395329599111169,Fri Apr 03 17:29:49 +0000 2020,"Oregon, USA",0,RT @ddale8: Once more: The president pushed ba...
9,778395329599111169,Fri Apr 03 17:31:10 +0000 2020,"Oregon, USA",0,RT @gtconway3d: “Kushner has succeeded at exac...


#### There are many attributes in a tweet which are not added, you can experiment with adding geolocation and many more interesting attributes to analyze data.

### Happy Coding